In [ ]:
from koselleck import *

In [ ]:
def get_paths_to_compare(ofnfn='data/data.all_local_dists.paths.csv',force=False):
    if not force and os.path.exists(ofnfn): 
        odf=read_df(ofnfn)
    else:
        dfpaths=get_model_paths_df(PATH_MODELS_BPO, 'dists.pkl').query('(period_end-period_start)==5')
        dfpaths['period']=[f'{x}-{y}' for x,y in zip(dfpaths.period_start, dfpaths.period_end)]
        o=[]
        for i1,row1 in tqdm(dfpaths.iterrows(), total=len(dfpaths)):
            for i2,row2 in dfpaths.iterrows():
                if row1.run!=row2.run: continue
                if i1>=i2: continue
                o+=[{
                    **dict((k+'1',v) for k,v in row1.items()),
                    **dict((k+'2',v) for k,v in row2.items())
                }]
        odf=pd.DataFrame(o)
        odf.to_csv(ofnfn,index=False)
    return odf.sort_values(['period_start1','period_start2','run1','run2'])

In [ ]:
dfpaths_cmp = get_paths_to_compare(force=True)
dfpaths_cmp

In [ ]:
# def do_gen_cross_model_dist(pathdf,progress=False,k=25):
#     row=pathdf.iloc[0]
#     dfdist1=read_df(row.path1)
#     dfdist2=read_df(row.path2)
    
#     words = list(set(dfdist1.columns) & set(dfdist2.columns))
#     dfdist1=dfdist1[words].loc[words]
#     dfdist2=dfdist2[words].loc[words]
    
#     o=[]
    
#     iterr = tqdm(words) if progress else words
#     for w in iterr:
#         neighb1=dfdist1[w].sort_values().iloc[1:k+1].index
#         neighb2=dfdist2[w].sort_values().iloc[1:k+1].index
#         metaneighb=list(set(neighb1)|set(neighb2))
        
#         vector1=[dfdist1[w].loc[wx] for wx in metaneighb]
#         vector2=[dfdist2[w].loc[wx] for wx in metaneighb]
        
#         csim=fastdist.cosine(vector1,vector2) # returns similarity not distane!!
#         dist=1-csim
        
#         o+=[{'word':w, 'dist_local':dist}]
#     return pd.DataFrame(o)
    

In [207]:
def do_gen_cross_model_dist(pathdf,progress=False,k=25):
    row=pathdf.iloc[0]
    dfdist1_orig=read_df(row.path1)
    
    o=[]
    iterr=tqdm(pathdf.path2,position=0) if progress else pathdf.path2
    for i2 in range(1,len(pathdf)):
        row2=pathdf.iloc[i2]
        
#     for path2 in pathdf.path2:
        dfdist2_orig=read_df(row2.path2)

        words = list(set(dfdist1_orig.columns) & set(dfdist2_orig.columns))
        dfdist1=dfdist1_orig[words].loc[words]
        dfdist2=dfdist2_orig[words].loc[words]

        iter2 = tqdm(words,position=0) if progress else words
        for w in iter2:
            neighb1=dfdist1[w].sort_values().iloc[1:k+1].index
            neighb2=dfdist2[w].sort_values().iloc[1:k+1].index
            metaneighb=list(set(neighb1)|set(neighb2))

            vector1=[dfdist1[w].loc[wx] for wx in metaneighb]
            vector2=[dfdist2[w].loc[wx] for wx in metaneighb]

            csim=fastdist.cosine(vector1,vector2) # returns similarity not distane!!
            dist=1-csim

            o+=[{
                'corpus2':row2.corpus2,'period2':row2.period2,'run2':row2.run2,
                'word':w, 'dist_local':dist}]
    return pd.DataFrame(o)
    

In [208]:
# odf=do_gen_cross_model_dist(dfpaths_cmp.iloc[:10], progress=True)
# odf.sort_values('dist_local')

In [209]:
def gen_cross_model_dist(dfpaths,lim=None,num_proc=1):
    return pmap_groups(
        do_gen_cross_model_dist,
#         dfpaths.iloc[:lim].groupby(['corpus1','corpus2','period1','period2','run1','run2']),
        dfpaths.iloc[:lim].groupby(['corpus1','period1','run1']),
        num_proc=num_proc,
    )

In [210]:
dfpaths_cmp_f = dfpaths_cmp#.query('run1<"run_02" & run2<"run_02"')
# dfpaths_cmp_f

In [ ]:
odf=gen_cross_model_dist(
    dfpaths_cmp_f,
    num_proc=5,
    lim=None
)
odf.to_pickle('data/data.all_local_dists.pkl')


  0%|          | 0/10 [1:07:38<?, ?it/s]

Mapping do_gen_cross_model_dist [x5]:  90%|█████████ | 789/875 [12:52:42<2:34:50, 108.03s/it]

In [ ]:
odf.sort_values('dist_local')